In [0]:
%tensorflow_version 1.x
# %keras_version 2.2.4
# !pip install tf-nightly-gpu-2.0-preview

TensorFlow 1.x selected.


In [0]:
import os, datetime
import numpy as np
import tensorflow as tf

# tf.compat.v1.disable_eager_execution()

from tensorflow.python.keras.models import *
from tensorflow.python.keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D, Add, Conv2DTranspose, BatchNormalization
from tensorflow.keras.optimizers import *
from tensorflow.python.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.losses import sparse_categorical_crossentropy, categorical_crossentropy
from tensorflow.python.keras import backend as K
from PIL import Image
from keras.utils.np_utils import to_categorical
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import random
import pdb
import glob

from tensorboardcolab import *
# import keras

Using TensorFlow backend.


In [0]:
# !mount --bind /content/drive/My\ Drive /content/MyDrive
# %cd /content/MyDrive

In [0]:
# !pip install tensorboardcolab

In [0]:
# logs_base_dir = "/content/MyDrive/Data_ML/Glo/tb_logs"
# os.makedirs(logs_base_dir, exist_ok=True)
# %load_ext tensorboard
# %reload_ext tensorboard
# %tensorboard --logdir {logs_base_dir} #--host localhost --port 8088
# tensorboard --logdir=data/ --host localhost --port 8088

In [0]:
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logs_base_dir, histogram_freq=1)

In [0]:
#downloaded = drive.CreateFile({'id':'https://drive.google.com/open?id=1h1pOR_x9mZS8Ayo-8zGJCxAPNTF_Poci'}) # replace the id with id of file you want to access

In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
class U_net(object):

	def __init__(self, img_rows = 1024, img_cols = 1024):

		self.img_rows = img_rows
		self.img_cols = img_cols
		self.current_folder_path = "/content/drive/My Drive/Data_ML/Glo"
		self.data_dir_path = self.current_folder_path + "/data"
		self.results_dir_path = self.current_folder_path + "/results"
		self.result_img_path = self.results_dir_path + "/mgs_mask_test.npy"
		self.train_dir_path = self.data_dir_path + "/train"
		self.img_dir_path = self.train_dir_path + "/image"
		self.label_dir_path = self.train_dir_path + "/label"
		self.img_org_dir = self.train_dir_path + "/image_org"
		self.img_play_dir = self.train_dir_path + "/image_play"
		self.label_play_dir = self.train_dir_path + "/label_play"
		self.label_org_dir = self.train_dir_path + "/label_org"
		self.test_img_play_dir = self.data_dir_path + "/test"
		self.npy_dir_path = self.current_folder_path + "/npydata"

		self.smooth = 1e-10

	def categorical_focal_loss_fixed(self, y_true, y_pred):
		"""
		:param y_true: A tensor of the same shape as `y_pred`
		:param y_pred: A tensor resulting from a softmax
		:return: Output tensor.
		"""
		gamma=2. 
		alpha=.25
		# Scale predictions so that the class probas of each sample sum to 1
		y_pred /= keras.sum(y_pred, axis=-1, keepdims=True)

		# Clip the prediction value to prevent NaN's and Inf's
		epsilon = keras.epsilon()
		y_pred = keras.clip(y_pred, epsilon, 1. - epsilon)

		# Calculate Cross Entropy
		cross_entropy = -y_true * keras.log(y_pred)

		# Calculate Focal Loss
		loss = alpha * keras.pow(1 - y_pred, gamma) * cross_entropy

		# Compute mean loss in mini_batch
		return keras.mean(loss, axis=1)
	# import keras.losses
	# keras.losses.custom_loss = custom_loss
	def my_categorical_crossentropy(self, output, target, from_logits=False):
			"""Categorical crossentropy between an output tensor and a target tensor.
			# Arguments
					output: A tensor resulting from a softmax
							(unless `from_logits` is True, in which
							case `output` is expected to be the logits).
					target: A tensor of the same shape as `output`.
					from_logits: Boolean, whether `output` is the
							result of a softmax, or is a tensor of logits.
			# Returns
					Output tensor.
			"""
			# Note: tf.nn.softmax_cross_entropy_with_logits
			# expects logits, Keras expects probabilities.
			if not from_logits:
					# scale preds so that the class probas of each sample sum to 1
					output /= tf.reduce_sum(output,
																	reduction_indices=len(output.get_shape()) - 1,
																	keep_dims=True)
					# manual computation of crossentropy
					epsilon = _to_tensor(_EPSILON, output.dtype.base_dtype)
					output = tf.clip_by_value(output, epsilon, 1. - epsilon)
					return - tf.reduce_sum(target * tf.log(output),
																	reduction_indices=len(output.get_shape()) - 1)
			else:
					return tf.nn.softmax_cross_entropy_with_logits(labels=target,
																													logits=output)

	def total_variation_loss(self, x):
		assert K.ndim(x) == 4
		# pdb.set_trace()
		img_nrows, img_ncols = x.shape[1:3]

		if K.image_data_format() == 'channels_first':
				a = K.square(
						x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, 1:, :img_ncols - 1])
				b = K.square(
						x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, :img_nrows - 1, 1:])
		else:
				a = K.square(
						x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
				b = K.square(
						x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
		return K.sum(K.pow(a + b, 1.25))
	
	def custom_loss (self, y_true, y_pred):
		return categorical_crossentropy(y_true, y_pred) + 0.0001 * self.total_variation_loss(y_pred)
		# return 0.0001 * self.total_variation_loss(y_pred)

	def my_unet_batch_norm(self, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, LearnRate):
		inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
		# ipdb.set_trace()
		c1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (inputs)
		b1 = BatchNormalization() (c1)
		# c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='glorot_uniform', padding='same') (c1)
		p1 = MaxPooling2D((2, 2)) (b1)

		c2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p1)
		b2 = BatchNormalization() (c2)
		# c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='glorot_uniform', padding='same') (c2)
		p2 = MaxPooling2D((2, 2)) (b2)

		c3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p2)
		b3 = BatchNormalization() (c3)
		# c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='glorot_uniform', padding='same') (c3)
		p3 = MaxPooling2D((2, 2)) (b3)

		c4 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p3)
		b4 = BatchNormalization() (c4)
		# c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='glorot_uniform', padding='same') (c4)
		p4 = MaxPooling2D((2, 2)) (b4)

		c5 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p4)
		b5 = BatchNormalization() (c5)
		p5 = MaxPooling2D(pool_size=(2, 2)) (b5)
		
		c6 = Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p5)
		drop6 = Dropout(0.5) (c6)
		# c6 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='glorot_uniform', padding='same') (drop6)
			
		u66 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same') (drop6)
		u66 = Add()([u66, c5])
		c66 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u66)
		# c66 = Dropout(0.1) (c66)
		# c66 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='glorot_uniform', padding='same') (c66)

		u6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same') (c66)
		u6 = Add()([u6, c4])
		c6 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u6)
		# c6 = Dropout(0.1) (c6)
		# c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='glorot_uniform', padding='same') (c6)

		u7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c6)
		u7 = Add()([u7, c3])
		c7 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u7)
		# c7 = Dropout(0.1) (c7)
		# c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='glorot_uniform', padding='same') (c7)

		u8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c7)
		u8 = Add()([u8, c2])
		c8 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u8)
		# c8 = Dropout(0.1) (c8)
		# c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='glorot_uniform', padding='same') (c8)

		u9 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c8)
		u9 = Add()([u9, c1])
		c9 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u9)
		# c9 = Dropout(0.1) (c9)
		# c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='glorot_uniform', padding='same') (c9)

		outputs = Conv2D(3, (1, 1), activation='softmax') (c9)

		model = Model(inputs=[inputs], outputs=[outputs])
		model.compile(optimizer = Adam(lr=LearnRate), loss = 'categorical_crossentropy' , metrics=['acc'])

		# model.compile(optimizer = Adam(lr=LearnRate), loss = self.custom_loss , metrics=['acc'])

		return model

	def train(self):
		# _, _, files = next(os.walk(self.img_dir_path))
		file_names_path = glob.glob(self.img_org_dir+ "/*.jpg")
		# files = glob.glob(data_dir_path)
		files = []
		for path in file_names_path:
			folders_list = path.split("/")
			files.append(folders_list[-1])
		items_no = len(files)
		random.shuffle(files)
		batch_size = 1

		model_path = self.current_folder_path + "/my_unet_total_variation.hdf5"
		# old_model_path = self.current_folder_path + "/my_unet_model_no_focal_2.hdf5"
		# model = load_model(old_model_path)

		# model = load_model(old_model_path, custom_objects={'categorical_focal_loss_fixed': self.categorical_focal_loss_fixed})
		# model = load_model(old_model_path, compile=False)

		# LearnRate = 0.0001
		# model.compile(optimizer = Adam(lr=LearnRate), loss = self.categorical_focal_loss_fixed , metrics=['acc'])

		model_checkpoint = ModelCheckpoint(model_path, monitor='loss',verbose=1, save_best_only=True)
		train_loops = 0
		# big_training = []
		# big_label = []

		# while len(files) >= batch_size:
		# 	train_items = files[:batch_size]
		# 	files = files[batch_size:]
		# 	train_batch = []
		# 	label_batch = []
		# for item_name in files:
		# 	# print(item_name)
		# 	item_path = self.img_org_dir + '/' + item_name
		# 	label_png_name = item_name[:-4] + '.png'
		# 	label_path = self.label_org_dir + '/' + label_png_name
		# 	item_arr = np.asarray(Image.open(item_path))
		# 	big_training.append(item_arr)
		# 	label_arr = np.asarray(Image.open(label_path))
		# 	label_arr = label_arr.copy()
		# 	label_arr[label_arr==170] = 1
		# 	label_arr[label_arr==250] = 2
		# 	# pdb.set_trace()
		# 	label_arr = to_categorical(label_arr, num_classes=3, dtype = 'uint8')
		# 	# pdb.set_trace()
		# 	big_label.append(label_arr)

		############################# UPLOADING TRAINING DATA FROM FOLDER###############################
		big_training_path = self.data_dir_path + '/all_images.npy'
		big_training = np.load(big_training_path)
		big_label_path = self.data_dir_path + '/all_labels_3cls.npy'
		big_label = np.load(big_label_path)
		#################################################################################################
		# big_training = np.asarray(big_training)
		# big_label = np.asarray(big_label)
		class_weight_manual = [5, 1, 1]
		# pdb.set_trace()

		# logdir = os.path.join(logs_base_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
		# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
		# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)
		# from tensorboardcolab import *
		# tb = TensorBoardColab(startup_waiting_time=12)

		model.fit(big_training, big_label, batch_size=4, nb_epoch=200, class_weight = class_weight_manual, verbose=1, shuffle=True, callbacks=[(model_checkpoint)])


In [0]:
# print(tf.__version__)
# print(keras.__version__)
# %tensorboard --logdir {logs_base_dir}

# from tensorboardcolab import *
# tb = TensorBoardColab()

# from tensorboard import notebook
# notebook.list()
# notebook.display(port=6006, height=1000)

In [0]:
myunet = U_net()
# jaccard_distance = myunet.jaccard_distance()
# jaccard_distance_loss = myunet.jaccard_distance_loss()
model = myunet.my_unet_batch_norm(1024, 1024, 3, 0.0001)
lista = ['']
print (lista)
myunet.train()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
['']
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2334 samples
Epoch 1/200
2332/2334 [============================>.] - ETA: 0s - loss: 4.7795 - acc: 0.6735
Epoch 00001: loss improved from inf to 4.77679, saving model to /content/drive/My Drive/Data_ML/Glo/my_unet_total_variation.hdf5
2334/2334 [==============================] - 962s 412ms/sample - loss: 4.7768 - acc: 0.6736
Epoch 2/200
2332/2334 [============================>.] - ETA: 0s - loss: 4.5174 - acc: 0.6803
Epoch 00002: loss improved from 4.77679 to 4.51559, saving model to /content/drive/My Drive/Data_ML/Glo/my_unet_total_variation.hdf5
2334/2334 [==============================] - 929s 398ms/sample - loss: 4.5156 - acc: 0.6803
Epoch 3/200
2332/2334 [============================>.] - ETA: 0s - loss: 3.8713 - acc: 0.7032
Epoch 00003: loss improved from 4.51559 to 3.87207, saving model to /c

KeyboardInterrupt: ignored